# Choosing a clustering algorithm

### Clustering algorithm selection

The best performers of each cluster algorithm type were:
    
K-Means with 3 clusters:    

ARI 0.6754724023467227 Silhouette 0.25709626983088596

    
DBSCAN with 3 eps regardles of min_samples:   

ARI 0.6327224213199801 Silhouette 0.23621420562267303

    
Gaussian with 2 clusters and diagonal covariance: 

ARI 0.9745646311690208 Silhouette 0.25090801755133996

    
Agglomerative Clustering with complete linkage and L1 affinity: 

ARI 0.9914710189786898 Silhoutte 0.2548496697185177

<b>Agglomerative edged out Gaussian in ARI and Sihouetter score, lets compare their actual cluster performance
side-by-side</b>

As shown below Gaussian clustered the data much more cleanly than the Aggomerative Clustering, only missing twelve observations.

##### Data Prep

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

In [3]:
#The dataset is quite large, and I am getting memory errors on this particular algorithm.
#I will split it randomnly.
df_features = df_features.sample(frac=0.05, random_state=763)

#We need to reset the index so the the algorithms below can loop through the dataframe
df_features = df_features.reset_index()

In [4]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [5]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

In [6]:
#The dataset is quite large, and I am getting memory errors on this particular algorithm.
#I will split it randomnly.
df_features = df_features.sample(frac=0.05, random_state=763)

#We need to reset the index so the the algorithms below can loop through the dataframe
df_features = df_features.reset_index()

### Data Prep with t-SNE dimensionality reduction

Elaboration of my dimensionality reduction configuration decision process is given in the Final Dimension Reduction Configuration notebook.

In [7]:
#Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

In [8]:
# Loading the data
X = df_features
y = df_features.is_canceled

# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

X_norm = normalize(X_std)

In [9]:
from sklearn.manifold import TSNE
import time

In [10]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=100, n_iter=300)
tsne_results = tsne.fit_transform(X_std)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 1978 samples in 0.017s...
[t-SNE] Computed neighbors for 1978 samples in 0.313s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1978
[t-SNE] Computed conditional probabilities for sample 1978 / 1978
[t-SNE] Mean sigma: 1.125884
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.465008
[t-SNE] KL divergence after 300 iterations: 0.498327
t-SNE done! Time elapsed: 9.236975193023682 seconds


## Aggolmerative

In [11]:
#Libraries
from sklearn.cluster import AgglomerativeClustering

In [12]:
# Defining the agglomerative clustering
agg_cluster = AgglomerativeClustering(linkage='complete', 
                                      affinity='l1',
                                      n_clusters=3)

# Fit model
A_clusters = agg_cluster.fit_predict(tsne_results)

In [13]:
plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(A_clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

In [14]:
A_clusters = np.array([0 if x == 1 else 1 if x==0 else 2 for x in A_clusters])

pd.crosstab(y, A_clusters)

col_0,0,1,2
is_canceled,,,
0,579,883,4
1,90,0,422


## Gaussian

In [15]:
#Libraries
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

In [16]:
# Defining the agglomerative clustering
gmm_cluster = GaussianMixture(n_components=2, random_state=123, covariance_type='diag')
_
# Fit model
G_clusters = gmm_cluster.fit_predict(X_std)

In [17]:
plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(G_clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

In [18]:
G_clusters = np.array([0 if x == 1 else 1 if x==0 else 2 for x in G_clusters])

pd.crosstab(y, G_clusters)

col_0,0,1
is_canceled,,
0,12,1454
1,512,0


### Summary

In this notebook I showed how I decided which clustering algorithm to use for Unsupervised Learning